In [2]:
import multiprocessing 
import gensim
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

C:\Users\Carla\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

bdtesis = client.BDTesis
documents2train = bdtesis.documents2train


In [18]:
#a minusculas
data = []
for x in bdtesis.documents2train.find():
    data.append(x['documento'])
datamin =[]
for x in (data):
    datamin.append(x.lower()) 
len(data)

4771

In [9]:
#limpieza de puntuación
import re
import nltk
import string
string.punctuation
datapunc = []
for x in datamin:
    datapunc.append(x.translate(str.maketrans('', '', string.punctuation)))


In [10]:
#retirar stopwords
from nltk.corpus import stopwords
stops = stopwords.words('spanish')
dataclean = []
for x in datapunc:
    palabras = x.split()
    resultwords  = [palabra for palabra in palabras if palabra.lower() not in stops]
    dataclean.append(' '.join(resultwords))
    


In [11]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(dataclean)]

In [12]:
#PARAMETROS
cores = multiprocessing.cpu_count()
words = 1
size = 300
context_window = 8
min_count = 19
epochs = 10
n_epochs = 10

In [13]:
n_epochs = 100
vec_size = 100
alpha = 0.025
model = Doc2Vec(size=vec_size, alpha=alpha,min_count=1, dm =0)
model.build_vocab(tagged_data)
%time model.train(tagged_data, total_examples=model.corpus_count, epochs=n_epochs)


model.save("./doc2vec_sbs_documents_pvdbow2")
  

C:\Users\Carla\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Wall time: 39.8 s


In [ ]:
model = Doc2Vec.load("./doc2vec_sbs_documents_pvdbow2")


In [14]:
model.most_similar(positive=['seguros'],topn=10)

C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('decidido', 0.39620912075042725),
 ('escrito', 0.34634608030319214),
 ('forma', 0.3419508934020996),
 ('permitidas', 0.3388301134109497),
 ('corredora', 0.3345121145248413),
 ('of', 0.31789931654930115),
 ('3°', 0.3164294362068176),
 ('administrar', 0.31448763608932495),
 ('establecidas', 0.3099898099899292),
 ('extendido', 0.3096519708633423)]

In [15]:
model.most_similar(positive=['soat'],topn=10)

C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('dependencias', 0.3360072672367096),
 ('precedente', 0.31341344118118286),
 ('proporcional', 0.311931312084198),
 ('emitidos', 0.2994183301925659),
 ('cambio7', 0.2992700934410095),
 ('adquisición', 0.29593825340270996),
 ('ambas', 0.29354527592658997),
 ('354°', 0.2904347777366638),
 ('nivel', 0.2875019907951355),
 ('compete', 0.2868596017360687)]